In [ ]:
import sys
sys.version

'3.7.12 (default, Sep 10 2021, 00:21:48) \n[GCC 7.5.0]'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls "drive/MyDrive/data"
!rm -rf "twitter-datasets"
!cp -r "drive/MyDrive/data" "twitter-datasets"

embeddings_trained.txt		       test_data.txt
embeddings_trained.txt.wv.vectors.npy  test_shuffled_full.tsv
embeddings.txt			       test_shuffled.tsv
embeddings.txt.syn1neg.npy	       train_shuffled_full.tsv
embeddings.txt.trainables.syn1neg.npy  train_shuffled.tsv
embeddings.txt.wv.vectors.npy


In [2]:
import torch

# Preparing Data

In [9]:
from torchtext.legacy import data

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(include_lengths = True)
LABEL = data.LabelField(dtype = torch.float)

In [10]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


train_data, test_data = data.TabularDataset.splits(
                                        path = 'twitter-datasets',
                                        train = 'train_shuffled.tsv',
                                        test = 'test_shuffled.tsv',
                                        format = 'tsv',
                                        fields = [("text", TEXT), ("label", LABEL)],
)

MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.twitter.27B.200d")
LABEL.build_vocab(train_data)
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

train_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, test_data), 
    batch_size = BATCH_SIZE,
    sort_key = lambda x: len(x.text),
    sort_within_batch = True,
    device = device)

.vector_cache\glove.twitter.27B.zip: 1.52GB [05:18, 4.78MB/s]                               
100%|█████████▉| 1193513/1193514 [01:25<00:00, 14018.66it/s]


Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABEL vocabulary: 2


# Using W2V

In [12]:
import multiprocessing
cores = multiprocessing.cpu_count()
print(cores)

8


In [15]:
sentences = []
with open('twitter-datasets/train_shuffled.tsv', 'r', encoding="utf-8") as f :
  line = f.readline()
  while line != "":
    sentences.append(line[:-3].split())
    line = f.readline()
  
with open('twitter-datasets/test_shuffled.tsv', 'r', encoding="utf-8") as f :
  line = f.readline()
  while line != "":
    sentences.append(line[:-3].split())
    line = f.readline()

In [16]:
# adding test data sentences

with open('twitter-datasets/test_data.txt', 'r') as f :
  line = f.readline()
  while line != "":
    line_arr = line.split(",")
    line = ""
    for l in line_arr[1:]:
      line += l
    sentences.append(line)
    line = f.readline()

In [17]:
len(sentences)

210000

In [18]:
from gensim.models import Word2Vec
w2v_model = Word2Vec(sentences, size=100, min_count=1, workers=cores-1, sg=0)

In [8]:
# Train Word Embeddings
w2v_model.train(sentences, total_examples=len(sentences), epochs=50)

(1519872229, 1972250350)

In [ ]:
w2v_model.save('embeddings_test.vec')

In [1]:
import gensim
print(gensim.__version__)

C:\Users\Arthur\AppData\Local\Programs\Python\Python37\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


3.6.0


In [ ]:
w2v_model = gensim.models.Word2Vec.load('twitter-datasets/embeddings.txt') # change for embeddings_trained # trained using different python version (3.8) leads to error
weights = torch.FloatTensor(w2v_model.wv.vectors)
embedding = torch.nn.Embedding.from_pretrained(weights)

# Preprocessing data

In [ ]:
import re
from nltk.metrics.distance import jaccard_distance
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, words
from nltk.stem import PorterStemmer
from nltk.util import ngrams
import random

import nltk
nltk.download('stopwords')
nltk.download('punkt')

correct_words = words.words()
stop_words = stopwords.words()
ps = PorterStemmer()

def clean_sentence(val):
    """remove chars that are not letters or numbers, downcase, 
    then remove stop words"""
    regex = re.compile('([^\s\w]|_)+')
    sentence = regex.sub('', val).lower()
    sentence = word_tokenize(sentence)
    for word in list(sentence):
        if word in stop_words:
            sentence.remove(word)
        else:
            words_list = [(jaccard_distance(set(ngrams(word, 2)), set(ngrams(w, 2))),w) for w in correct_words if w[0]==word[0]]
            try:
                word_best = sorted(words_list, key = lambda val:val[0])[0][1]
            except:
                word_best = word
            sentence[sentence.index(word)] = ps.stem(word_best)
    sentence = " ".join(sentence)
    return sentence

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
with open('twitter-datasets/train_shuffled.tsv', 'r') as f :
  with open('twitter-datasets/train_shuffled_processed.tsv', 'w') as f2 :
    line = f.readline()
    while line != "" :
      line = f.readline() # [:-3] ~ removes /t <int> /t
      print(line[:-3])
      f2.write(clean_sentence(line[:-3]) + line[-3:] )

#9221 we have had good times in the past & i miss my lil pothead
sanyo rl - 4920 directsync - mobile charging kit ( high current output ) ( uk outlet plug the directsync - mobile c ... <url>
<user> i thought i answered on fb that i work :p sorrry ! ! ! ya i cant today ( (


ZeroDivisionError: ignored

# Model

In [14]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.rnn = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout)
        
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text, text_lengths):
        
        #text = [sent len, batch size]
        
        embedded = self.dropout(self.embedding(text))
        
        #embedded = [sent len, batch size, emb dim]
        
        #pack sequence
        # lengths need to be on CPU!
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.to('cpu'))
        
        packed_output, (hidden, cell) = self.rnn(packed_embedded)
        
        #unpack sequence
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)

        #output = [sent len, batch size, hid dim * num directions]
        #output over padding tokens are zero tensors
        
        #hidden = [num layers * num directions, batch size, hid dim]
        #cell = [num layers * num directions, batch size, hid dim]
        
        #concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers
        #and apply dropout
        
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
                
        #hidden = [batch size, hid dim * num directions]
            
        return self.fc(hidden)

In [15]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 200
HIDDEN_DIM = 128
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = RNN(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT, 
            PAD_IDX)

In [16]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 5,733,841 trainable parameters


In [17]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()
model = model.to(device)
criterion = criterion.to(device)

In [18]:
device

device(type='cpu')

In [19]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [20]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        text, text_lengths = batch.text
        
        predictions = model(text, text_lengths).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [21]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            text, text_lengths = batch.text
            
            predictions = model(text, text_lengths).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [22]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

# Training

In [24]:
N_EPOCHS = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 14m 17s
	Train Loss: 0.321 | Train Acc: 85.47%


# Predictions

In [31]:
def predict_sentiment(model, sentence):
    model.eval()
    tokenized = sentence.split()
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    prediction = torch.sigmoid(model(tensor, length_tensor))
    return prediction.item()

In [33]:
print(vars(train_data.examples[0]))

{'text': ['<user>', 'waaay', 'back', 'when', '...', 'i', 'miss', 'it'], 'label': '0'}


In [34]:
predict_sentiment(model, "This film is terrible")

0.986739993095398

In [ ]:
!ls twitter-datasets

test_data.txt  test_shuffled.tsv  train_shuffled.tsv


In [35]:
x = 0
predictions = []
ids = []
with open('twitter-datasets/test_data.txt', 'r') as f :
  line = f.readline()
  while line != "" :
    line_arr = line.split(",")
    line = ""
    for l in line_arr[1:]:
      line += l
    y_pred = predict_sentiment(model, line)
    if y_pred > 0.5:
      y_pred = -1
    else:
      y_pred = 1
    predictions.append(y_pred)
    ids.append(line_arr[0])
    line = f.readline()
    x += 1
    if x % 1000 == 0:
      print(f"sentence {x}")

sentence 1000
sentence 2000
sentence 3000
sentence 4000
sentence 5000
sentence 6000
sentence 7000
sentence 8000
sentence 9000
sentence 10000


In [36]:
import csv
def create_csv_submission(ids, y_pred, name):
    """
    Creates an output file in .csv format for submission to Kaggle or AIcrowd
    Arguments: ids (event ids associated with each prediction)
               y_pred (predicted class labels)
               name (string name of .csv output file to be created)
    """
    with open(name, 'w') as csvfile:
        fieldnames = ['Id', 'Prediction']
        writer = csv.DictWriter(csvfile, delimiter=",", fieldnames=fieldnames)
        writer.writeheader()
        for r1, r2 in zip(ids, y_pred):
            writer.writerow({'Id':int(r1),'Prediction':int(r2)})

create_csv_submission(ids, predictions, "submission.csv")

In [ ]:
!ls

drive  embeddings.txt  sample_data  submission.csv  twitter-datasets


In [ ]:
from google.colab import files
files.download('submission.csv') 